In [1]:
import taichi as ti
import cv2
import numpy as np
import os
import shutil

# ====================================================================================
# GPU PARALLELISM EXPLANATION:
# Taichi compiles @ti.kernel functions to GPU shaders that execute in parallel:
# 1. Data-oriented design: Fields (x, v, F, etc.) are allocated in GPU memory
# 2. Parallel loops: 'for p in x' launches 9000+ GPU threads simultaneously
# 3. Memory coalescing: Structured grid access patterns optimize memory bandwidth
# 4. Hierarchical parallelism: Particles and grid nodes computed concurrently
# 5. SIMD efficiency: GPU processes particles in lockstep (same instructions)
# ====================================================================================

# Initialize Taichi with GPU backend and enable kernel profiling
ti.init(arch=ti.gpu, kernel_profiler=True, device_memory_GB=1.0)

# Simulation parameters
quality = 1  # Increase for higher resolution
n_particles, n_grid = 9000 * quality**2, 128 * quality
dx, inv_dx = 1 / n_grid, float(n_grid)
dt = 1e-4 / quality
p_vol, p_rho = (dx * 0.5) ** 2, 1
p_mass = p_vol * p_rho
E, nu = 0.1e4, 0.2  # Young's modulus and Poisson's ratio
mu_0, lambda_0 = E / (2 * (1 + nu)), E * nu / ((1 + nu) * (1 - 2 * nu))  # Lame parameters

# GPU data containers (allocated in VRAM)
x = ti.Vector.field(2, dtype=float, shape=n_particles)  # position
v = ti.Vector.field(2, dtype=float, shape=n_particles)  # velocity
C = ti.Matrix.field(2, 2, dtype=float, shape=n_particles)  # affine velocity field
F = ti.Matrix.field(2, 2, dtype=float, shape=n_particles)  # deformation gradient
material = ti.field(dtype=int, shape=n_particles)  # material id
Jp = ti.field(dtype=float, shape=n_particles)  # plastic deformation
grid_v = ti.Vector.field(2, dtype=float, shape=(n_grid, n_grid))  # grid node velocity
grid_m = ti.field(dtype=float, shape=(n_grid, n_grid))  # grid node mass

# Quadratic B-spline interpolation weights
@ti.func
def n(x):
    abs_x = ti.abs(x)
    return (abs_x < 0.5) * (0.75 - abs_x ** 2) + \
           (abs_x >= 0.5) * (abs_x < 1.5) * (0.5 * (1.5 - abs_x) ** 2)

# Compute interpolation weights
@ti.func
def quad_weights(fx):
    wx = ti.Vector([n(fx[0] - 0.0), n(fx[0] - 1.0), n(fx[0] - 2.0)])
    wy = ti.Vector([n(fx[1] - 0.0), n(fx[1] - 1.0), n(fx[1] - 2.0)])
    return wx, wy

# Main simulation step (GPU kernel)
@ti.kernel
def substep():
    # Reset grid (parallel over grid nodes)
    for i, j in grid_m:
        grid_v[i, j] = [0, 0]
        grid_m[i, j] = 0
    
    # Particle-to-Grid transfer (P2G) - parallel over particles
    for p in x:  
        # Compute base grid coordinates
        base = (x[p] * inv_dx - 0.5).cast(int)
        fx = x[p] * inv_dx - base.cast(float)
        wx, wy = quad_weights(fx)
        
        # Update deformation gradient
        F[p] = (ti.Matrix.identity(float, 2) + dt * C[p]) @ F[p]
        
        # Material-specific hardening
        h = ti.exp(10 * (1.0 - Jp[p]))
        if material[p] == 1:  # jelly
            h = 0.3
        mu, la = mu_0 * h, lambda_0 * h
        if material[p] == 0:  # liquid
            mu = 0.0
        
        # SVD-based plasticity model
        U, sig, V = ti.svd(F[p])
        for d in ti.static(range(2)):
            sig[d, d] = ti.max(sig[d, d], 1e-6)
        
        J = 1.0
        for d in ti.static(range(2)):
            new_sig = sig[d, d]
            if material[p] == 2:  # Snow plasticity
                new_sig = ti.min(ti.max(sig[d, d], 1 - 2.5e-2), 1 + 4.5e-3)
            Jp[p] *= sig[d, d] / new_sig
            sig[d, d] = new_sig
            J *= new_sig
        
        # Reconstruct deformation gradient
        if material[p] == 0:
            F[p] = ti.Matrix.identity(float, 2) * ti.sqrt(J)
        elif material[p] == 2:
            F[p] = U @ sig @ V.transpose()
        
        # Compute stress
        stress = 2 * mu * (F[p] - U @ V.transpose()) @ F[p].transpose() + \
                 ti.Matrix.identity(float, 2) * la * J * (J - 1)
        stress = (-dt * p_vol * 4 * inv_dx * inv_dx) * stress
        affine = stress + p_mass * C[p]
        
        # Scatter to grid (3x3 neighborhood)
        for i, j in ti.static(ti.ndrange(3, 3)):
            offset = ti.Vector([i, j])
            dpos = (offset.cast(float) - fx) * dx
            weight = wx[i] * wy[j]
            grid_v[base + offset] += weight * (p_mass * v[p] + affine @ dpos)
            grid_m[base + offset] += weight * p_mass
    
    # Grid update (parallel over grid nodes)
    for i, j in grid_m:
        if grid_m[i, j] > 0:
            grid_v[i, j] = (1 / grid_m[i, j]) * grid_v[i, j]
            grid_v[i, j][1] -= dt * 50  # gravity
            
            # Boundary conditions
            if i < 3 and grid_v[i, j][0] < 0: grid_v[i, j][0] = 0
            if i > n_grid - 3 and grid_v[i, j][0] > 0: grid_v[i, j][0] = 0
            if j < 3 and grid_v[i, j][1] < 0: grid_v[i, j][1] = 0
            if j > n_grid - 3 and grid_v[i, j][1] > 0: grid_v[i, j][1] = 0
    
    # Grid-to-Particle transfer (G2P) - parallel over particles
    for p in x:  
        base = (x[p] * inv_dx - 0.5).cast(int)
        fx = x[p] * inv_dx - base.cast(float)
        wx, wy = quad_weights(fx)
        new_v = ti.Vector.zero(float, 2)
        new_C = ti.Matrix.zero(float, 2, 2)
        
        for i, j in ti.static(ti.ndrange(3, 3)):
            dpos = ti.Vector([i, j]).cast(float) - fx
            g_v = grid_v[base + ti.Vector([i, j])]
            weight = wx[i] * wy[j]
            new_v += weight * g_v
            new_C += 4 * inv_dx * weight * g_v.outer_product(dpos)
        
        v[p], C[p] = new_v, new_C
        x[p] += dt * v[p]  # advection

# Initialize particles
group_size = n_particles // 3

@ti.kernel
def initialize():
    for i in range(n_particles):
        x[i] = [
            ti.random() * 0.2 + 0.3 + 0.10 * (i // group_size),
            ti.random() * 0.2 + 0.05 + 0.32 * (i // group_size),
        ]
        material[i] = i // group_size  # 0: fluid, 1: jelly, 2: snow
        v[i] = ti.Matrix([0, 0])
        F[i] = ti.Matrix([[1, 0], [0, 1]])
        Jp[i] = 1

# Video recording function
def record_video(output_dir, output_file, fps=60, resolution=(512, 512)):
    """Compile frames to video using OpenCV"""
    if not os.path.exists(output_dir):
        print(f"Directory {output_dir} not found!")
        return
    
    frames = sorted([f for f in os.listdir(output_dir) if f.startswith("frame_") and f.endswith(".png")])
    if not frames:
        print("No frames found for video compilation")
        return
    
    # Determine video resolution from first frame
    first_frame = cv2.imread(os.path.join(output_dir, frames[0]))
    h, w, _ = first_frame.shape
    size = (w, h) if resolution is None else resolution
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_file, fourcc, fps, size)
    
    print(f"Compiling {len(frames)} frames to {output_file}...")
    for frame_file in frames:
        img = cv2.imread(os.path.join(output_dir, frame_file))
        if resolution is not None:
            img = cv2.resize(img, resolution)
        video.write(img)
    
    video.release()
    print(f"Video saved to {output_file}")
    print(f"Video specs: {size[0]}x{size[1]} @ {fps} FPS")

# Main function with video recording
def main():
    # Create output directories
    output_dir = "simulation_frames"
    video_path = "physics_simulation.mp4"
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize simulation
    initialize()
    gui = ti.GUI("Multi-Material Physics Simulator", res=512, background_color=0x112F41)
    
    # Video recording parameters
    max_frames = 300  # 5 seconds at 60 FPS
    frame_count = 0
    
    print("Starting simulation with video recording...")
    print(f"Particles: {n_particles}, Grid: {n_grid}x{n_grid}")
    print("Materials: 0=Fluid (blue), 1=Jelly (red), 2=Snow (white)")
    
    # Main simulation loop
    while not gui.get_event(ti.GUI.ESCAPE, ti.GUI.EXIT) and frame_count < max_frames:
        # Advance physics
        for s in range(int(2e-3 // dt)):
            substep()
        
        # Render particles
        gui.circles(
            x.to_numpy(),
            radius=1.5,
            palette=[0x068587, 0xED553B, 0xEEEEF0],  # Blue, Red, White
            palette_indices=material,
        )
        
        # Save frame
        frame_file = os.path.join(output_dir, f"frame_{frame_count:06d}.png")
        gui.show(frame_file)
        frame_count += 1
        print(f"Frame {frame_count}/{max_frames} rendered", end='\r')
    
    # Compile video
    print("\nSimulation complete. Compiling video...")
    record_video(output_dir, video_path, fps=60, resolution=(1024, 1024))
    
    # Cleanup options (uncomment to delete frames after video creation)
    # shutil.rmtree(output_dir)
    # print(f"Deleted {frame_count} temporary frames")
    
    # Print GPU performance stats
    ti.profiler.print_kernel_profiler_info()

if __name__ == "__main__":
    main()

[Taichi] version 1.7.3, llvm 15.0.1, commit 5ec301be, win, python 3.10.16
[Taichi] Starting on arch=cuda
Starting simulation with video recording...
Particles: 9000, Grid: 128x128
Materials: 0=Fluid (blue), 1=Jelly (red), 2=Snow (white)
Frame 300/300 rendered
Simulation complete. Compiling video...
Compiling 300 frames to physics_simulation.mp4...
Video saved to physics_simulation.mp4
Video specs: 1024x1024 @ 60 FPS
Kernel Profiler(count, default) @ CUDA on NVIDIA GeForce RTX 4060 Laptop GPU
[      %     total   count |      min       avg       max   ] Kernel name
----------------------------------------------------------------------------
[ 57.98%   0.770 s   5700x |    0.019     0.135     1.149 ms] substep_c76_0_kernel_1_range_for
[ 21.60%   0.287 s   5700x |    0.004     0.050     3.977 ms] substep_c76_0_kernel_0_range_for
[ 11.56%   0.154 s   5700x |    0.006     0.027     1.945 ms] substep_c76_0_kernel_3_range_for
[  7.71%   0.102 s   5700x |    0.004     0.018     0.195 ms] subst